In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flowers102-dataset/102flowers.tgz
/kaggle/input/flowers102-dataset/imagelabels.mat


In [2]:
import tensorflow as tf
from tensorflow.keras import layers as tfl
import numpy as np
import os
import tarfile
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.io

2025-10-04 19:04:07.304553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759604647.546214      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759604647.613045      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset_path = "/kaggle/input/flowers102-dataset"
images_path = os.path.join(dataset_path, "102flowers.tgz")
labels_path = os.path.join(dataset_path, "imagelabels.mat")

In [4]:
if not os.path.exists("102flowers"):
    tar = tarfile.open(images_path)
    tar.extractall()
    tar.close()
    print("Images extracted successfully!")

Images extracted successfully!


In [5]:
labels = scipy.io.loadmat(labels_path)['labels'][0]

image_dir = "jpg" if os.path.exists("jpg") else "102flowers/jpg"
image_files = sorted(os.listdir(image_dir))

images = []
image_labels = []

for i, file in enumerate(image_files[:2000]):
    img_path = os.path.join(image_dir, file)
    img = Image.open(img_path).resize((64, 64))
    img = np.array(img)
    if img.shape == (64, 64, 3):  
        images.append(img)
        image_labels.append(labels[i] - 1)

images = np.array(images) / 255.0
image_labels = np.array(image_labels)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, image_labels, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (1600, 64, 64, 3)
Test shape: (400, 64, 64, 3)


In [24]:
def flower102model(input_shape=(64,64,3), num_classes=102):
    model = tf.keras.Sequential([
        tf.keras.layers.ZeroPadding2D(padding=3, input_shape=input_shape),
        tf.keras.layers.Conv2D(filters=20,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=20,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=20,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=48,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=48,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=48,kernel_size=(3,3), strides=(1,1), padding='same'),
        tf.keras.layers.BatchNormalization(axis=3),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(200,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(100,activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation='softmax')
        
    ])
    
    return model

In [25]:
model = flower102model()

In [26]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_2                │ (None, 70, 70, 3)      │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 70, 70, 20)     │           560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 70, 70, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 70, 70, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 70, 70, 20)     │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 70, 70, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 70, 70, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 70, 70, 20)     │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 70, 70, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 70, 70, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 35, 35, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 35, 35, 48)     │         8,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 35, 35, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_15 (ReLU)                 │ (None, 35, 35, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 35, 35, 48)     │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 35, 35, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_16 (ReLU)                 │ (None, 35, 35, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 35, 35, 48)     │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 35, 35, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_17 (ReLU)                 │ (None, 35, 35, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 2,863,874 (10.92 MB)

 Trainable params: 2,863,466 (10.92 MB)

 Non-trainable params: 408 (1.59 KB)

In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 92ms/step - accuracy: 0.0645 - loss: 6.0524 - val_accuracy: 0.1950 - val_loss: 4.0498
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.1625 - loss: 3.0074 - val_accuracy: 0.1225 - val_loss: 3.2039
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.2308 - loss: 2.4895 - val_accuracy: 0.0925 - val_loss: 2.9627
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.2770 - loss: 2.2909 - val_accuracy: 0.1075 - val_loss: 2.8029
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3084 - loss: 2.1937 - val_accuracy: 0.0875 - val_loss: 2.8588
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3479 - loss: 2.0203 - val_accuracy: 0.0675 - val_loss: 2.9412
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3325 - loss: 2.0148 - val_accuracy: 0.0700 - val_loss: 3.3060
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3729 - loss: 1.9490 - val_accuracy: 0